In [1]:
# from proteinbert import load_pretrained_model
# from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

2023-08-30 16:22:26.907662: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-30 16:22:27.309030: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'


In [8]:
df = pd.read_csv('mutated.csv')

In [19]:
df = df.drop(2063).reset_index(drop=True)

In [20]:
df.tail(50)

,Unnamed: 0,UniProt ID,Mutants,Protein Sequence,delta_G,mutated_sequence
2044,2053,Q5NV88,H8P,NFMLTQPHSVSESPGKTVTISCTRSSGSIASNYVQWYQQRPGSSPT...,4.50,NFMLTQPPSVSESPGKTVTISCTRSSGSIASNYVQWYQQRPGSSPT...
2045,2054,Q5NV88,H8S,NFMLTQPHSVSESPGKTVTISCTRSSGSIASNYVQWYQQRPGSSPT...,4.50,NFMLTQPSSVSESPGKTVTISCTRSSGSIASNYVQWYQQRPGSSPT...
2046,2055,Q70YQ6,wild-type,MIVSMIAALANNRVIGLDNKMPWHLPAELQLFKRATLGKPIVMGRN...,-3.45,MIVSMIAALANNRVIGLDNKMPWHLPAELQLFKRATLGKPIVMGRN...
2047,2056,Q75QQ5,wild-type,MNSYQNKNEYEILNAPSNNTNMPNRYPFANDPNAMMKNGNYKDWLD...,23.10,MNSYQNKNEYEILNAPSNNTNMPNRYPFANDPNAMMKNGNYKDWLD...
2048,2057,Q75QQ5,I189L,MNSYQNKNEYEILNAPSNNTNMPNRYPFANDPNAMMKNGNYKDWLD...,22.78,MNSYQNKNEYEILNAPSNNTNMPNRYPFANDPNAMMKNGNYKDWLD...
2049,2058,Q75QQ5,L175V,MNSYQNKNEYEILNAPSNNTNMPNRYPFANDPNAMMKNGNYKDWLD...,12.61,MNSYQNKNEYEILNAPSNNTNMPNRYPFANDPNAMMKNGNYKDWLD...
2050,2059,Q7LZZ4,wild-type,EEPLPDRAVPIYVAGFLALPRGPFFVGLIAVQLLRDRSLVAERRRQ...,12.70,EEPLPDRAVPIYVAGFLALPRGPFFVGLIAVQLLRDRSLVAERRRQ...
2051,2060,Q7LZZ4,C350S,EEPLPDRAVPIYVAGFLALPRGPFFVGLIAVQLLRDRSLVAERRRQ...,9.30,EEPLPDRAVPIYVAGFLALPRGPFFVGLIAVQLLRDRSLVAERRRQ...
2052,2061,Q7LZZ4,G288A,EEPLPDRAVPIYVAGFLALPRGPFFVGLIAVQLLRDRSLVAERRRQ...,9.30,EEPLPDRAVPIYVAGFLALPRGPFFVGLIAVQLLRDRSLVAERRRQ...
2053,2062,Q7LZZ4,E291D,EEPLPDRAVPIYVAGFLALPRGPFFVGLIAVQLLRDRSLVAERRRQ...,7.20,EEPLPDRAVPIYVAGFLALPRGPFFVGLIAVQLLRDRSLVAERRRQ...


In [21]:
df.loc[2063]

Unnamed: 0                                                       2077
UniProt ID                                                     Q8Z9H3
Mutants                                                     wild-type
Protein Sequence    MADRNLRDLLAPWVAGLPARELREMTLDSRVAAAGDLFVAVVGHQA...
delta_G                                                          3.27
mutated_sequence    MADRNLRDLLAPWVAGLPARELREMTLDSRVAAAGDLFVAVVGHQA...
Name: 2063, dtype: object

In [ ]:
protein_sequences = df['mutated_sequence']


In [ ]:
max_sequence_length = -1
max_sequence_index = -1

for index, seq in enumerate(protein_sequences):
    if len(seq) > max_sequence_length:
        max_sequence_length = len(seq)
        max_sequence_index = index

# Print the results
print("Maximum Sequence Length:", max_sequence_length)
print("Index of Sequence with Maximum Length:", max_sequence_index)
print("Sequence with Maximum Length:", protein_sequences[max_sequence_index])


In [ ]:
max_sequence_length = max(len(seq) for seq in protein_sequences)


In [ ]:
max_sequence_length

In [22]:
# Assuming 'df' is your DataFrame
protein_sequences = df['mutated_sequence']
delta_g_values = df['delta_G']

# Define the amino acids and their order
amino_acids = "ACDEFGHIKLMNPQRSTVWY"

def one_hot_encode_sequence(sequence):
    # Create a dictionary to map each amino acid to an index
    aa_to_index = {aa: i for i, aa in enumerate(amino_acids)}
    
    # Initialize an array of zeros with the shape (sequence_length, number_of_amino_acids)
    sequence_length = len(sequence)
    one_hot_encoded = np.zeros((sequence_length, len(amino_acids)), dtype=int)
    
    # Set the appropriate index to 1 for each amino acid in the sequence
    for i, aa in enumerate(sequence):
        one_hot_encoded[i, aa_to_index[aa]] = 1
    
    return one_hot_encoded

# One-hot encode all the protein sequences in the DataFrame
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assuming you have a list of one-hot encoded sequences
encoded_sequences = [one_hot_encode_sequence(seq) for seq in protein_sequences]

#Find the maximum sequence length
max_sequence_length = max(len(seq) for seq in encoded_sequences)

# Pad sequences to the maximum length
padded_sequences = pad_sequences(encoded_sequences, maxlen=max_sequence_length, padding='post', truncating='post', dtype='float32')

# Convert the padded_sequences list to a numpy array
X = np.array(padded_sequences)

# Convert the delta_g_values to a numpy array
y = np.array(delta_g_values)
del protein_sequences
del delta_g_values
del encoded_sequences
del padded_sequences


In [ ]:
# Split the data into 70% training and 30% test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3,random_state=42)

# # Split the remaining data (30%) into 10% validation and 20% test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.67, random_state=42)

# # Print the shapes of the datasets
print("Training data shapes:", X_train.shape, y_train.shape)
print("Validation data shapes:", X_val.shape, y_val.shape)
print("Test data shapes:", X_test.shape, y_test.shape)


In [23]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

In [25]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

# Set the input shape and number of classes (1 for regression)
input_shape = (X_train.shape[1:])
num_classes = 1

# Create the model architecture
def create_model():
    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='linear'))
    
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
    return model

# Initialize lists to store R2 scores for each fold
r2_scores = []
mse = []
# Create a KFold cross-validator
num_folds = 3
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Iterate through cross-validation folds
for train_idx, val_idx in kf.split(X_train):
    # Split the data into train and validation sets for this fold
    X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
    y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]
    print(X_fold_train.shape)
    # Create a new instance of the model for each fold
    model = create_model()
   
    
    # Train the model on the current fold's training data
    model.fit(X_fold_train, y_fold_train, epochs=100, batch_size=64, validation_data=(X_fold_val, y_fold_val))
    
    # Evaluate the model on the current fold's validation data
    y_fold_val_pred = model.predict(X_fold_val)
    mse_i = mean_squared_error(y_fold_val, y_fold_val_pred)
    r2 = r2_score(y_fold_val, y_fold_val_pred)
    print(r2)
    r2_scores.append(r2)
    mse.append(mse_i)
    

# Calculate the mean and standard deviation of R2 scores
mean_r2 = np.mean(r2_scores)
std_r2 = np.std(r2_scores)
mse = np.mean(mse)
print(f"Mean R2 across {num_folds} folds: {mean_r2:.4f}")
print(f"Standard Deviation of R2: {std_r2:.4f}")
print(f"mean of mean square error of R2: {std_r2:.4f}")


(744, 7073, 20)
Epoch 1/100
12/12 [==============================] - 2s 131ms/step - loss: 22.9050 - val_loss: 12.1013
Epoch 2/100
12/12 [==============================] - 0s 42ms/step - loss: 12.1844 - val_loss: 10.5143
Epoch 3/100
12/12 [==============================] - 0s 42ms/step - loss: 11.4881 - val_loss: 9.3475
Epoch 4/100
12/12 [==============================] - 0s 42ms/step - loss: 10.2871 - val_loss: 8.5789
Epoch 5/100
12/12 [==============================] - 0s 41ms/step - loss: 9.1405 - val_loss: 7.5712
Epoch 6/100
12/12 [==============================] - 0s 41ms/step - loss: 8.1884 - val_loss: 6.5328
Epoch 7/100
12/12 [==============================] - 0s 41ms/step - loss: 6.4102 - val_loss: 6.0243
Epoch 8/100
12/12 [==============================] - 0s 41ms/step - loss: 5.3728 - val_loss: 5.5561
Epoch 9/100
12/12 [==============================] - 0s 42ms/step - loss: 4.4237 - val_loss: 4.9081
Epoch 10/100
12/12 [==============================] - 0s 41ms/step - loss: 4.

In [24]:
# num_folds = 3

# # Initialize lists to store R2 scores for each fold
# r2_scores = []
# mse_scores = []

# # Create a KFold cross-validator
# kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# # Iterate through cross-validation folds
# for train_val_idx, test_idx in kf.split(X):
#     X_train_val, X_test = X[train_val_idx], X[test_idx]
#     y_train_val, y_test = y[train_val_idx], y[test_idx]

#     # Split train_val into train and validation sets
#     X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

#     # Set the input shape and number of classes (1 for regression)
#     input_shape = (X_train.shape[1], X_train.shape[2])
#     num_classes = 1
    
#     # Create a new instance of the model for each fold
#     model = Sequential()
#     model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape))
#     model.add(MaxPooling1D(pool_size=2))
#     model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#     model.add(MaxPooling1D(pool_size=2))
#     model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
#     model.add(MaxPooling1D(pool_size=2))
#     model.add(Flatten())
#     model.add(Dense(256, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Dense(num_classes, activation='linear'))
    
#     model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
    
#     # Train the model on the current fold's training data
#     model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val))
    
#     # Evaluate the model on the current fold's validation data
#     y_val_pred = model.predict(X_val)
#     r2 = r2_score(y_val, y_val_pred)
#     mse = mean_squared_error(y_val, y_val_pred)
#     r2_scores.append(r2)
#     mse_scores.append(mse)

# # Calculate the mean and standard deviation of R2 scores
# mean_r2 = np.mean(r2_scores)
# std_r2 = np.std(r2_scores)
# mean_mse = np.mean(mse_scores)
# print(f"Mean R2 across {num_folds} folds: {mean_r2:.4f}")
# print(f"Standard Deviation of R2: {std_r2:.4f}")
# print(f"Mean of Mean Squared Error: {mean_mse:.4f}")



2023-08-30 16:24:01.575460: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-30 16:24:01.589529: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-30 16:24:01.589680: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Epoch 1/100


2023-08-30 16:24:02.771122: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700
2023-08-30 16:24:03.068262: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


18/18 [==============================] - 3s 95ms/step - loss: 23.6201 - val_loss: 9.0007
Epoch 2/100
18/18 [==============================] - 1s 36ms/step - loss: 14.5516 - val_loss: 8.0877
Epoch 3/100
18/18 [==============================] - 1s 36ms/step - loss: 11.8413 - val_loss: 9.6039
Epoch 4/100
18/18 [==============================] - 1s 35ms/step - loss: 10.8861 - val_loss: 10.3337
Epoch 5/100
18/18 [==============================] - 1s 36ms/step - loss: 9.4944 - val_loss: 6.7484
Epoch 6/100
18/18 [==============================] - 1s 35ms/step - loss: 7.9475 - val_loss: 6.0481
Epoch 7/100
18/18 [==============================] - 1s 36ms/step - loss: 7.0157 - val_loss: 5.7321
Epoch 8/100
18/18 [==============================] - 1s 36ms/step - loss: 7.2908 - val_loss: 6.4487
Epoch 9/100
18/18 [==============================] - 1s 36ms/step - loss: 6.4802 - val_loss: 5.6956
Epoch 10/100
18/18 [==============================] - 1s 35ms/step - loss: 6.1136 - val_loss: 5.4855
Epoch 

In [ ]:
from sklearn.model_selection import KFold
import numpy as np

# Define the number of folds
n_splits = 3  # You can adjust this as needed

# Initialize the KFold cross-validator
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

data_splits = {}

# Iterate over each fold
for count, (train_idx, val_idx) in enumerate(kf.split(X), start=1):
    X_fold_train, X_fold_val = X[train_idx], X[val_idx]
    y_fold_train, y_fold_val = y[train_idx], y[val_idx]
    
    # Store the datasets in a dictionary
    data_splits[f'fold_{count}'] = {
        'X_train': X_fold_train,
        'X_val': X_fold_val,
        'y_train': y_fold_train,
        'y_val': y_fold_val
    }


In [ ]:
del X
del y

In [ ]:
# Access data for the first fold
fold_num = 1
fold_data = data_splits[f'fold_{fold_num}']
X_fold_train = fold_data['X_train']
X_fold_val = fold_data['X_val']
y_fold_train = fold_data['y_train']
y_fold_val = fold_data['y_val']
input_shape = (X_fold_train.shape[1:])
num_classes = 1
# Now you can use X_fold_train, y_fold_train for training and X_fold_val, y_fold_val for validation
# Train and validate your model using this data
def create_model():
    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='linear'))
    
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
    return model

# Initialize lists to store R2 scores for each fold
r2_scores = []

# Create a KFold cross-validator
num_folds = 3
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Iterate through cross-validation folds

    # Split the data into train and validation sets for this fold

print(X_fold_train.shape)
# Create a new instance of the model for each fold
model = create_model()


# Train the model on the current fold's training data
model.fit(X_fold_train, y_fold_train, epochs=100, batch_size=64)

# Evaluate the model on the current fold's validation data
y_fold_val_pred = model.predict(X_fold_val)
r2 = r2_score(y_fold_val, y_fold_val_pred)
mse_i = mean_squared_error(y_fold_val, y_fold_val_pred)
print(r2)
print(mse_i)
r2_scores.append(r2)
del model

In [ ]:
# Access data for the first fold
fold_num = 2
fold_data = data_splits[f'fold_{fold_num}']
X_fold_train = fold_data['X_train']
X_fold_val = fold_data['X_val']
y_fold_train = fold_data['y_train']
y_fold_val = fold_data['y_val']
input_shape = (X_fold_train.shape[1:])
num_classes = 1
# Now you can use X_fold_train, y_fold_train for training and X_fold_val, y_fold_val for validation
# Train and validate your model using this data
def create_model():
    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='linear'))
    
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
    return model

# Initialize lists to store R2 scores for each fold

# Create a KFold cross-validator
num_folds = 3
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Iterate through cross-validation folds

    # Split the data into train and validation sets for this fold

print(X_fold_train.shape)
# Create a new instance of the model for each fold
model = create_model()


# Train the model on the current fold's training data
model.fit(X_fold_train, y_fold_train, epochs=100, batch_size=64, validation_data=(X_fold_val, y_fold_val))

# Evaluate the model on the current fold's validation data
y_fold_val_pred = model.predict(X_fold_val)
mse_i = mean_squared_error(y_fold_val, y_fold_val_pred)
print(r2)
print(mse_i)
r2_scores.append(r2)
del model

In [ ]:
# Access data for the first fold
fold_num = 3
fold_data = data_splits[f'fold_{fold_num}']
X_fold_train = fold_data['X_train']
X_fold_val = fold_data['X_val']
y_fold_train = fold_data['y_train']
y_fold_val = fold_data['y_val']
input_shape = (X_fold_train.shape[1:])
num_classes = 1
# Now you can use X_fold_train, y_fold_train for training and X_fold_val, y_fold_val for validation
# Train and validate your model using this data
def create_model():
    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='linear'))
    
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
    return model

# Initialize lists to store R2 scores for each fold

# Create a KFold cross-validator
num_folds = 3
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Iterate through cross-validation folds

    # Split the data into train and validation sets for this fold

print(X_fold_train.shape)
# Create a new instance of the model for each fold
model = create_model()


# Train the model on the current fold's training data
model.fit(X_fold_train, y_fold_train, epochs=20, batch_size=5, validation_data=(X_fold_val, y_fold_val))

# Evaluate the model on the current fold's validation data
y_fold_val_pred = model.predict(X_fold_val)
mse_i = mean_squared_error(y_fold_val, y_fold_val_pred)
print(r2)
print(mse_i)
r2_scores.append(r2)
del model

In [ ]:
import numpy as np

r2_scores = ['0.6112126816674337', '0.490500579327808', '0.4435895074279347']

# Convert the R2 scores to floats
r2_scores_float = [float(score) for score in r2_scores]

# Calculate mean and standard deviation
mean_r2 = np.mean(r2_scores_float)
std_r2 = np.std(r2_scores_float)

print(f"Mean R2 across {len(r2_scores_float)} folds: {mean_r2:.4f}")
print(f"Standard Deviation of R2: {std_r2:.4f}")
